In [1]:
import sys
import torch
import numpy as np
import torch
import traci
from sumolib import checkBinary
import torch
import sys
import io
from contextlib import redirect_stdout
import os


if 'SUMO_HOME' in os.environ:
    print('SUMO_HOME found')
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))

sumoBinary = checkBinary('sumo-gui')
# sumoBinary = checkBinary('sumo')
roadNetwork = "./config/osm.sumocfg"
sumoCmd = [sumoBinary, "-c", roadNetwork, "--start", "--quit-on-end"]
# use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

SUMO_HOME found
Using device: cuda


In [2]:
def intervehicleConnectivity(threshold = None):
    xs = []
    ys = []
    for vehicle in traci.vehicle.getIDList():
        x, y = traci.vehicle.getPosition(vehicle)
        xs.append(x)
        ys.append(y)
    xs = torch.tensor(xs, dtype=torch.float32).to(device).view(-1,1)
    ys = torch.tensor(ys, dtype=torch.float32).to(device).view(-1,1)
    intervehicle_distances = torch.sqrt((xs - xs.t())**2 + (ys - ys.t())**2)
    if threshold is not None:
        # make the distances 1 if less than the threshold, 0 otherwise
        intervehicle_distances = torch.where(intervehicle_distances < threshold, torch.ones_like(intervehicle_distances), torch.zeros_like(intervehicle_distances))
    return intervehicle_distances, xs, ys

In [3]:
def randomTrips(dur=1000, density=12):
    os.system("python $SUMO_HOME/tools/randomTrips.py -n config/osm.net.xml.gz -r config/osm.passenger.trips.xml -e " + str(dur) + " -l --insertion-density=" + str(density))

def shouldContinueSim():
    numVehicles = traci.simulation.getMinExpectedNumber()
    return True if numVehicles > 0 else False

def restart(sumoCmd):
    with io.StringIO() as buf, redirect_stdout(buf):
        try:
            traci.close()
        except:
            pass
        traci.start(sumoCmd)

def close():
    traci.close()

In [4]:
import copy

class Knowledges:
    def __init__(self):
        self.knowledges = {}
        self.delays = {}
    
    def add_observations(self, vehicles, observed_vehicles):
        for vehicle, visibility in zip(vehicles, observed_vehicles):
            if vehicle not in self.knowledges:
                self.knowledges[vehicle] = []
                self.delays[vehicle] = 0
            self.knowledges[vehicle].append(int(visibility))
            if visibility == 0:
                self.delays[vehicle] += 1
            else:
                self.delays[vehicle] = 0
    
    def merge_knowledges(self, new_knowledges, new_delays):
        prev_missing, prev_delay, _, _ = self.evaluate_knowledge()
        for vehicle, visibility in new_knowledges.items():
            if vehicle not in self.knowledges:
                self.knowledges[vehicle] = copy.deepcopy(visibility)
                self.delays[vehicle] = new_delays[vehicle]
            else:
                for i in range(1, len(self.knowledges[vehicle])+1):
                    if i > len(visibility):
                        break
                    self.knowledges[vehicle][-i] = visibility[-i] | self.knowledges[vehicle][-i]
                self.delays[vehicle] = min(self.delays[vehicle], new_delays[vehicle])
        new_missing, new_delay, _, _ = self.evaluate_knowledge()
        return copy.deepcopy(self.knowledges), copy.deepcopy(self.delays), prev_missing - new_missing, prev_delay - new_delay

    def get_knowledges(self):
        return copy.deepcopy(self.knowledges)
    
    def get_delays(self):
        return copy.deepcopy(self.delays)
    
    def evaluate_knowledge(self, large_delay_threshold=10):
        observed = 0
        delay = 0
        large_delay = 0
        num_vehicles = len(self.knowledges)
        whole_missing = 0
        total = 0
        for vehicle, visibility in self.knowledges.items():
            observed += sum(visibility) 
            total += len(visibility)
            delay += self.delays[vehicle]
            if self.delays[vehicle] >= large_delay_threshold:
                large_delay += 1
            if sum(visibility) == 0:
                whole_missing += len(visibility)
        return 1-(observed / total), delay / num_vehicles, large_delay / num_vehicles, whole_missing / total

class Beacon:
    def __init__(self):
        self.trace = []
    
    def update_trace(self, hop):
        self.trace.append(hop)
        if len(self.trace) > 10:
            self.trace = self.trace[-10:]

class Vehicle:
    def __init__(self):
        self.sent = {}
        self.received = 0

    def step(self):
        self.received = 0
        for vehicle, lag in self.sent.items():
            self.sent[vehicle] += 1
        self.sent = {vehicle: lag for vehicle, lag in self.sent.items() if lag < 10}
    
    def receive(self):
        self.received += 1

    # lag: the time since the last communication with a neighbor
    def select(self, neighbors):
        max_lag = 0
        selected = None
        for neighbor in neighbors:
            if neighbor not in self.sent:
                selected = neighbor
                break
            if self.sent[neighbor] > max_lag:
                max_lag = self.sent[neighbor]
                selected = neighbor
        return selected
    
    def send(self, selected_neighbor):
        self.sent[selected_neighbor] = 0


In [11]:
# find the maximum action space
restart(sumoCmd)
max_action_space = 0
vehicle_knowledges = {}
total_actions = 0

total_missing_gain = 0
total_delay_gain = 0
# total_large_delay = 0
step = 0
max_action_space = 0
vehicle_records = {}
vehicel_carrying_beacons = {}
total_states = 0
total_missing = 0
total_delay = 0
total_large_delay = 0
total_whole_missing = 0

max_received = 0

while shouldContinueSim():
    step += 1
    if step > 1100:
        close()
        break
    traci.simulationStep()
    ids = traci.vehicle.getIDList()
    connectivity, xs, ys = intervehicleConnectivity(800)
    # minus the diagonal to get the action space, vehicle cannot send to itself
    action_spaces = connectivity.to("cpu") - torch.eye(connectivity.size(0))
    
    # update the observed vinicity for each vehicle
    for i, vehicle in enumerate(ids):
        if vehicle not in vehicle_knowledges:
            vehicle_knowledges[vehicle] = Knowledges()
        if vehicle not in vehicle_records:
            vehicle_records[vehicle] = Vehicle()
        vehicle_knowledges[vehicle].add_observations(ids, connectivity[i])

    for i, vehicle in enumerate(ids):
        # get non-zero indices except the diagonal
        non_zero_indices = np.where(action_spaces[i] == 1)[0]
        neighbors = [ids[j] for j in non_zero_indices]
        if len(neighbors) > 0:
            selected_neighbor = vehicle_records[vehicle].select(neighbors)
            if selected_neighbor is None:
                continue
            vehicle_records[vehicle].send(selected_neighbor)
            vehicle_records[selected_neighbor].receive()
            _, _, missing_gain, delay_gain = vehicle_knowledges[selected_neighbor].merge_knowledges(vehicle_knowledges[vehicle].get_knowledges(), vehicle_knowledges[vehicle].get_delays())
            total_missing_gain += missing_gain
            total_delay_gain += delay_gain
            total_actions += 1
            
            # echo
            vehicle_records[selected_neighbor].send(vehicle)
            vehicle_records[vehicle].receive()
            _, _, missing_gain, delay_gain = vehicle_knowledges[vehicle].merge_knowledges(vehicle_knowledges[selected_neighbor].get_knowledges(), vehicle_knowledges[selected_neighbor].get_delays())
            total_missing_gain += missing_gain
            total_delay_gain += delay_gain
            total_actions += 1
            
    for vehicle in vehicle_records.values():
        vehicle.step()
    for i, vehicle in enumerate(ids):
        total_states += 1
        missing, delay, large_delay, whole_missing = vehicle_knowledges[vehicle].evaluate_knowledge()
        total_whole_missing += whole_missing
        total_missing += missing
        total_delay += delay
        total_large_delay += large_delay
print("Average missing: ", total_missing / total_states)
print("Average delay: ", total_delay / total_states)
print("Average large delay: ", total_large_delay / total_states)
print("Average whole missing: ", total_whole_missing / total_states)
print("Max received: ", max_received)

# print("Average missing gain: ", total_missing_gain / total_actions)
# print("Average delay gain: ", total_delay_gain / total_actions)

Average missing:  0.3316355493253351
Average delay:  17.435119780178187
Average large delay:  0.413359304284807
Average whole missing:  0.2129991377030123
Max received:  9


In [39]:
num_vehicles = len(vehicle_knowledges)
total_missing = 0
total_delay = 0
step = 0
max_delay = 0 
for vehicle, knowledge in vehicle_knowledges.items():
    m, d, l, w = knowledge.evaluate_knowledge() 
    total_missing += m
    total_delay += d
    max_delay = max(max_delay, l)
print("Average missing: ", total_missing/num_vehicles)
print("Average delay: ", total_delay/num_vehicles)
print("Max delay: ", max_delay)

Average missing:  0.1941228882964494
Average delay:  22.090666637289097
Max delay:  0.8974358974358975
